In [ ]:
%%capture
!pip install openai

In [ ]:
import pandas as pd
import openai

In [ ]:
df = pd.read_csv('gpt_best_prompts_summs.csv')

In [ ]:
openai.api_key = #insert API key here

In [ ]:
def get_score(prompt):
  response = openai.ChatCompletion.create(
              model="gpt-4",
              messages=[
                  {"role": "system", "content": ""},
                  {"role": "user", "content": prompt},
              ],
              temperature=0.1,
              max_tokens=1,
              timeout=45,
              n=1,
              stop=None
          )

  return response['choices'][0]['message']['content']

In [ ]:
def get_prompt(gt_sum, model_sum):
  prompt = """
Task: \nIm giving you two medical summaries. The first summary is the ground-truth one. The second summary is an AI model generated one.\
 First, see if the ground-truth summary has any medical terms. If it doesn't, output 'None'. If it does, then please compare the\
 AI generated summary with the ground-truth summary to see if its medical terms are accurate to the ones in the ground-truth summary\
 and if the medical facts are accurate and similar/same as the ones in the ground-truth summary, output a score from 1 to 5 where 1\
 is the majority/all medical terms and facts are not the same and incorrect, 3 is about half, and 5 is the\
 majority/all. \n\nGround-truth summary: \n{ground_truth} \n\nAI model summary: \n{model} \n\nScore:
  """.format(ground_truth=gt_sum, model=model_sum)
  return prompt

In [ ]:
models = ["text-ada-001", "text-babbage-001","text-curie-001", "text-davinci-003", "gpt-3.5-turbo", "gpt-4"]

In [ ]:
# comment this cell if using df instead of sub_df
sub_df = df.head(5).copy()

In [ ]:
# replace all "sub_df" with "df"
new_columns = sub_df.apply(
    lambda row: pd.Series(
        {f'gpt-4_factuality_score_{mod}': get_score(get_prompt(row['section_text'], row[mod]))
         for mod in models}
    ),
    axis=1
)

sub_df = pd.concat([sub_df, new_columns], axis=1)

In [ ]:
#can be changed to "df" to view the resulting df
sub_df

In [ ]:
#change from "sub_df" to "df"
sub_df.to_csv("gpt4_factuality_scores.csv")